In [47]:
# Importing Libraries required for NLP
!pip install tflearn 
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
# Importing Libraries needed for Tensorflow processing
import tensorflow as tf   
import json           
import random
import tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
# intents has all the datafrom intents.json file
with open("/content/intents_1.json") as json_data: 
    intents = json.load(json_data)      

# New section

# New section

In [49]:
# Empty collection to store after performing operations
no_of_words=[]
no_of_documents = []
no_of_classes = []


# to ignoring unrealted marking.
ignore = ["?"]

# looping through intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # word tokenizer is used to tokenize the words
        wd = nltk.word_tokenize(pattern) 
        
        # appending to the empty word collection
        no_of_words.extend(wd) 
        
        # appending intents file
        no_of_documents.append((wd, intent["tag"]))
        
        # Adding tags to class collection
        if intent["tag"] not in no_of_classes:      
            no_of_classes.append(intent["tag"])  

In [50]:
#iterating to word collection and ignoring unrealted words
no_of_words = [stemmer.stem(w.lower()) for w in no_of_words if w not in ignore]  
no_of_words = sorted(list(set(no_of_words)))  #Removing all Duplicates in no_of_words[]

#deleting CLasses whic are Duplicate
no_of_classes = sorted(list(set(no_of_classes)))

#Printing length of lists we formed
print(len(no_of_documents),"Documents \n")
print(len(no_of_classes),"Classes \n")
print(len(no_of_words), "Stemmed Words ")

283 Documents 

26 Classes 

193 Stemmed Words 


In [51]:
# training Data
training = []
output = []

#to get output we have created empty array
output_empty = [0] * len(no_of_classes)

#Creating Training set and bag of words for each sentence
for doc in no_of_documents:
    bag = [] #Initialising empty bag of words

    pattern_words = doc[0] #Storing list of tokenized words for the documents[] tp pattern_words
    #print(pattern_words)
    
    #Again Stemming each word from pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    #Creating bag of words array
    for w in no_of_words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #It will give output 1 for curent tag and 0 for all other tags
    output_row = list(output_empty)
    output_row[no_of_classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [52]:
random.shuffle(training) #Suffling the data or features
training = np.array(training) #Converting training data into numpy array

#Creating Training Lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [53]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
#tf.reset_default_graph() #Reset Underlying Graph data

#Building our own Neural Network

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)

#Defining Model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

#Now we have setup model, now we need to train that model by fitting data into it by model.fit()
#n_epoch is the number of times that model will se our data during training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model

Training Step: 35999  | total loss: 0.00003 | time: 0.120s
| Adam | epoch: 1000 | loss: 0.00003 - acc: 1.0000 -- iter: 280/283
Training Step: 36000  | total loss: 0.00003 | time: 0.123s
| Adam | epoch: 1000 | loss: 0.00003 - acc: 1.0000 -- iter: 283/283
--


In [54]:
#Importing pickle module
import pickle

#Dumping training data by using dump() and writing it into training_data in binary mode
pickle.dump({"words":no_of_words, "classes":no_of_classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [55]:
#Restoring all data structure
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [56]:
with open("/content/intents_1.json") as json_data:
    intents = json.load(json_data)  #Loading our json_data

In [57]:
# Loading the saved model
model.load("./model.tflearn") #Loading training model which we saved

In [58]:
#Cleaning User Input
def clean_up_sentence(sentence):
    
    # Tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Stemming each word from the user's input
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

#Returning bag of words array: 0 or 1 or each word in the bag that exists in as we have declared in above lines
def bow(sentence, words, show_details=False):
    
    #Tokenizing the user input
    sentence_words = clean_up_sentence(sentence)
    
    #Generating bag of words from the sentence that user entered
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [59]:
#Adding some context to the conversation for better results.

context = {} #Create a dictionary to hold user's context

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    #Generating probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    #Filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    #Sorting by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    #If we have a classification then find the matching intent tag
    if results:
        
        #Loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                
                #Find a tag matching the first result
                if i['tag'] == results[0][0]:
                    
                    #Set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        #A random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [60]:
response("who made you")

I was developed by Nishanth


In [61]:
response("where is the University located")

University of North Texas,1155 Union Cir, Denton, TX 76203


In [62]:
response("size of unt")

Sad to see you go :(


In [63]:
response("college buses")

you can find more info in https://transportation.unt.edu/
